In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import torch
torch.rand(10)
import torch.nn as nn
import torch.nn.functional as F

print(torch.cuda.is_available())
print(torch.cuda.get_device_name())
print(torch.cuda.current_device())

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

True
TITAN RTX
0
Using device: cuda

TITAN RTX
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [10]:
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
%cat /home/user01/data_ssd/Talha/yolo/paprika_y5/data.yaml

train: /home/user01/data_ssd/Talha/yolo/paprika_y5/train/images
val: /home/user01/data_ssd/Talha/yolo/paprika_y5/test/images

nc: 6
names: ["blossom_end_rot", "graymold","powdery_mildew","spider_mite","spotting_disease", "snails_and_slugs"]

In [3]:
%cat /home/user01/data_ssd/Talha/yolo/ScaledYOLOv4/models/yolov4-csp.yaml

# parameters
nc: 80  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple

# anchors
anchors:
  - [12,16, 19,36, 40,28]  # P3/8
  - [36,75, 76,55, 72,146]  # P4/16
  - [142,110, 192,243, 459,401]  # P5/32

# yolov4-csp backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [32, 3, 1]],  # 0
   [-1, 1, Conv, [64, 3, 2]],  # 1-P1/2
   [-1, 1, Bottleneck, [64]],
   [-1, 1, Conv, [128, 3, 2]],  # 3-P2/4
   [-1, 2, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 5-P3/8
   [-1, 8, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 7-P4/16
   [-1, 8, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]], # 9-P5/32
   [-1, 4, BottleneckCSP, [1024]],  # 10
  ]

# yolov4-csp head
# na = len(anchors[0])
head:
  [[-1, 1, SPPCSP, [512]], # 11
   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [8, 1, Conv, [256, 1, 1]], # route backbone P4
   [[-1, -2], 1, Concat, [1]],
   [-1, 2, Bott

In [ ]:
# paprika_y5
# cells_v5
%cd /home/user01/data_ssd/Talha/yolo/ScaledYOLOv4/

!python train.py --img 416 --batch 16 --epochs 100 \
        --data '/home/user01/data_ssd/Talha/yolo/paprika_y5/data.yaml' \
        --cfg ./models/yolov4-csp.yaml \
        --weights '' \
        --name yolov4-csp-results  \
        --cache

/home/user01/data_ssd/Talha/yolo/ScaledYOLOv4
Using CUDA device0 _CudaDeviceProperties(name='TITAN RTX', total_memory=24217MB)

Namespace(adam=False, batch_size=16, bucket='', cache_images=True, cfg='./models/yolov4-csp.yaml', data='/home/user01/data_ssd/Talha/yolo/paprika_y5/data.yaml', device='', epochs=100, evolve=False, global_rank=-1, hyp='data/hyp.scratch.yaml', img_size=[416, 416], local_rank=-1, logdir='runs/', multi_scale=False, name='yolov4-csp-results', noautoanchor=False, nosave=False, notest=False, rect=False, resume=False, single_cls=False, sync_bn=False, total_batch_size=16, weights='', world_size=1)
Start Tensorboard with "tensorboard --logdir runs/", view at http://localhost:6006/
Hyperparameters {'lr0': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'giou': 0.05, 'cls': 0.5, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.5, 'scale': 0.5, 'shear': 0.0

      8/99     6.05G   0.05581   0.05514   0.00652    0.1175        22       416
               Class      Images     Targets           P           R      mAP@.5
                 all    2.03e+03    6.27e+03       0.367       0.718       0.592        0.25

     Epoch   gpu_mem      GIoU       obj       cls     total   targets  img_size
      9/99     6.05G   0.05483   0.05429  0.005872     0.115        28       416
               Class      Images     Targets           P           R      mAP@.5
                 all    2.03e+03    6.27e+03       0.342       0.758       0.624       0.262

     Epoch   gpu_mem      GIoU       obj       cls     total   targets  img_size
     10/99     6.05G   0.05387   0.05361  0.005435    0.1129        37       416
               Class      Images     Targets           P           R      mAP@.5
                 all    2.03e+03    6.27e+03       0.408       0.784       0.678       0.293

     Epoch   gpu_mem      GIoU       obj       cls     total   targets

               Class      Images     Targets           P           R      mAP@.5
                 all    2.03e+03    6.27e+03       0.497        0.86       0.806       0.422

     Epoch   gpu_mem      GIoU       obj       cls     total   targets  img_size
     33/99     6.05G   0.04384   0.04653  0.002662   0.09303        21       416
               Class      Images     Targets           P           R      mAP@.5
                 all    2.03e+03    6.27e+03       0.517       0.859       0.803       0.427

     Epoch   gpu_mem      GIoU       obj       cls     total   targets  img_size
     34/99     6.05G    0.0435   0.04702  0.002473   0.09299        21       416
               Class      Images     Targets           P           R      mAP@.5
                 all    2.03e+03    6.27e+03       0.504       0.865       0.811       0.434

     Epoch   gpu_mem      GIoU       obj       cls     total   targets  img_size
     35/99     6.05G   0.04357   0.04698  0.002423   0.09297        19